In [3]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [14]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)
    
    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    
    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
 
            noise = np.random.normal(0, 1, (batch_size, 100))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                
    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

In [12]:
X_train.shape
np.expand_dims(X_train, axis=3).shape

(60000, 28, 28, 1)

In [15]:
gan = GAN()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

In [1]:
import tensorflow as tf

In [2]:
#tf.__version__

'1.4.0'

In [10]:
gan.train(epochs=30000, batch_size=32, sample_interval=200)

0 [D loss: 0.578485, acc.: 79.69%] [G loss: 0.807983]
1 [D loss: 0.607593, acc.: 67.19%] [G loss: 0.866895]
2 [D loss: 0.607518, acc.: 68.75%] [G loss: 0.846238]
3 [D loss: 0.619513, acc.: 65.62%] [G loss: 0.867225]
4 [D loss: 0.615869, acc.: 70.31%] [G loss: 0.893293]
5 [D loss: 0.601033, acc.: 68.75%] [G loss: 0.854860]
6 [D loss: 0.597852, acc.: 75.00%] [G loss: 0.872814]
7 [D loss: 0.604460, acc.: 67.19%] [G loss: 0.870994]
8 [D loss: 0.616566, acc.: 73.44%] [G loss: 0.871101]
9 [D loss: 0.594228, acc.: 70.31%] [G loss: 0.880580]
10 [D loss: 0.608080, acc.: 71.88%] [G loss: 0.824969]
11 [D loss: 0.584594, acc.: 71.88%] [G loss: 0.868259]
12 [D loss: 0.613784, acc.: 73.44%] [G loss: 0.829522]
13 [D loss: 0.628831, acc.: 65.62%] [G loss: 0.861724]
14 [D loss: 0.577425, acc.: 75.00%] [G loss: 0.815699]
15 [D loss: 0.613820, acc.: 68.75%] [G loss: 0.892621]
16 [D loss: 0.597551, acc.: 70.31%] [G loss: 0.876586]
17 [D loss: 0.603861, acc.: 70.31%] [G loss: 0.883820]
18 [D loss: 0.642198

150 [D loss: 0.586259, acc.: 64.06%] [G loss: 0.884077]
151 [D loss: 0.634612, acc.: 70.31%] [G loss: 0.860130]
152 [D loss: 0.616045, acc.: 67.19%] [G loss: 0.812938]
153 [D loss: 0.595015, acc.: 73.44%] [G loss: 0.873203]
154 [D loss: 0.585887, acc.: 67.19%] [G loss: 0.923797]
155 [D loss: 0.628352, acc.: 68.75%] [G loss: 0.858117]
156 [D loss: 0.542135, acc.: 78.12%] [G loss: 0.878281]
157 [D loss: 0.617923, acc.: 62.50%] [G loss: 0.835947]
158 [D loss: 0.592919, acc.: 71.88%] [G loss: 0.803412]
159 [D loss: 0.599573, acc.: 67.19%] [G loss: 0.825985]
160 [D loss: 0.548005, acc.: 70.31%] [G loss: 0.888340]
161 [D loss: 0.646507, acc.: 53.12%] [G loss: 0.901513]
162 [D loss: 0.586912, acc.: 67.19%] [G loss: 0.911104]
163 [D loss: 0.567303, acc.: 70.31%] [G loss: 0.879327]
164 [D loss: 0.576776, acc.: 71.88%] [G loss: 0.874930]
165 [D loss: 0.567827, acc.: 76.56%] [G loss: 0.923030]
166 [D loss: 0.581957, acc.: 75.00%] [G loss: 0.905408]
167 [D loss: 0.569398, acc.: 75.00%] [G loss: 0.

299 [D loss: 0.544753, acc.: 79.69%] [G loss: 0.868356]
300 [D loss: 0.555939, acc.: 76.56%] [G loss: 0.924636]
301 [D loss: 0.540680, acc.: 73.44%] [G loss: 0.954062]
302 [D loss: 0.593601, acc.: 71.88%] [G loss: 0.909219]
303 [D loss: 0.613562, acc.: 67.19%] [G loss: 0.879879]
304 [D loss: 0.597294, acc.: 65.62%] [G loss: 0.904190]
305 [D loss: 0.553730, acc.: 76.56%] [G loss: 0.948341]
306 [D loss: 0.614344, acc.: 75.00%] [G loss: 0.886441]
307 [D loss: 0.599894, acc.: 62.50%] [G loss: 0.945430]
308 [D loss: 0.554390, acc.: 79.69%] [G loss: 1.019186]
309 [D loss: 0.602394, acc.: 76.56%] [G loss: 0.943133]
310 [D loss: 0.703747, acc.: 57.81%] [G loss: 0.820432]
311 [D loss: 0.532536, acc.: 71.88%] [G loss: 0.881670]
312 [D loss: 0.593220, acc.: 65.62%] [G loss: 0.948435]
313 [D loss: 0.609022, acc.: 68.75%] [G loss: 0.985651]
314 [D loss: 0.629802, acc.: 67.19%] [G loss: 0.891637]
315 [D loss: 0.570622, acc.: 79.69%] [G loss: 0.947761]
316 [D loss: 0.568529, acc.: 73.44%] [G loss: 0.

448 [D loss: 0.580197, acc.: 73.44%] [G loss: 0.883521]
449 [D loss: 0.565642, acc.: 78.12%] [G loss: 1.005137]
450 [D loss: 0.547338, acc.: 75.00%] [G loss: 1.030252]
451 [D loss: 0.544059, acc.: 78.12%] [G loss: 1.008024]
452 [D loss: 0.534497, acc.: 79.69%] [G loss: 1.034047]
453 [D loss: 0.685243, acc.: 56.25%] [G loss: 1.009089]
454 [D loss: 0.576304, acc.: 67.19%] [G loss: 0.991230]
455 [D loss: 0.571149, acc.: 70.31%] [G loss: 0.956356]
456 [D loss: 0.619311, acc.: 70.31%] [G loss: 0.845608]
457 [D loss: 0.560181, acc.: 75.00%] [G loss: 0.943071]
458 [D loss: 0.573948, acc.: 67.19%] [G loss: 0.999516]
459 [D loss: 0.627063, acc.: 62.50%] [G loss: 1.058201]
460 [D loss: 0.558488, acc.: 67.19%] [G loss: 1.018156]
461 [D loss: 0.613155, acc.: 65.62%] [G loss: 0.972782]
462 [D loss: 0.545944, acc.: 68.75%] [G loss: 0.943833]
463 [D loss: 0.618697, acc.: 62.50%] [G loss: 1.015956]
464 [D loss: 0.539002, acc.: 81.25%] [G loss: 0.914031]
465 [D loss: 0.591914, acc.: 75.00%] [G loss: 0.

596 [D loss: 0.595578, acc.: 67.19%] [G loss: 0.934543]
597 [D loss: 0.522255, acc.: 75.00%] [G loss: 0.925847]
598 [D loss: 0.573373, acc.: 68.75%] [G loss: 0.913492]
599 [D loss: 0.602867, acc.: 71.88%] [G loss: 1.066616]
600 [D loss: 0.513288, acc.: 82.81%] [G loss: 0.970758]
601 [D loss: 0.509766, acc.: 78.12%] [G loss: 1.011644]
602 [D loss: 0.619534, acc.: 67.19%] [G loss: 1.002296]
603 [D loss: 0.588841, acc.: 73.44%] [G loss: 0.980807]
604 [D loss: 0.599189, acc.: 68.75%] [G loss: 0.966270]
605 [D loss: 0.627747, acc.: 60.94%] [G loss: 1.006783]
606 [D loss: 0.621975, acc.: 64.06%] [G loss: 0.984170]
607 [D loss: 0.575336, acc.: 73.44%] [G loss: 1.030343]
608 [D loss: 0.558420, acc.: 76.56%] [G loss: 0.901448]
609 [D loss: 0.493867, acc.: 81.25%] [G loss: 0.998364]
610 [D loss: 0.585728, acc.: 70.31%] [G loss: 0.951741]
611 [D loss: 0.578984, acc.: 71.88%] [G loss: 0.934761]
612 [D loss: 0.626947, acc.: 60.94%] [G loss: 0.990798]
613 [D loss: 0.572850, acc.: 67.19%] [G loss: 0.

...

29369 [D loss: 0.623845, acc.: 64.06%] [G loss: 0.976924]
29370 [D loss: 0.640731, acc.: 59.38%] [G loss: 1.129834]
29371 [D loss: 0.698768, acc.: 51.56%] [G loss: 1.027729]
29372 [D loss: 0.620188, acc.: 68.75%] [G loss: 1.107786]
29373 [D loss: 0.652865, acc.: 65.62%] [G loss: 1.183505]
29374 [D loss: 0.625866, acc.: 62.50%] [G loss: 1.068509]
29375 [D loss: 0.648054, acc.: 64.06%] [G loss: 0.982980]
29376 [D loss: 0.642537, acc.: 59.38%] [G loss: 1.037395]
29377 [D loss: 0.585560, acc.: 65.62%] [G loss: 1.134490]
29378 [D loss: 0.699209, acc.: 60.94%] [G loss: 1.223495]
29379 [D loss: 0.547634, acc.: 76.56%] [G loss: 1.157006]
29380 [D loss: 0.648894, acc.: 57.81%] [G loss: 1.057480]
29381 [D loss: 0.730705, acc.: 54.69%] [G loss: 1.171598]
29382 [D loss: 0.550930, acc.: 71.88%] [G loss: 1.204961]
29383 [D loss: 0.663405, acc.: 65.62%] [G loss: 1.061590]
29384 [D loss: 0.725593, acc.: 46.88%] [G loss: 1.089964]
29385 [D loss: 0.592834, acc.: 75.00%] [G loss: 1.183974]
29386 [D loss:

29513 [D loss: 0.586909, acc.: 62.50%] [G loss: 1.007345]
29514 [D loss: 0.661058, acc.: 62.50%] [G loss: 1.091737]
29515 [D loss: 0.705390, acc.: 59.38%] [G loss: 1.082955]
29516 [D loss: 0.586329, acc.: 71.88%] [G loss: 1.186730]
29517 [D loss: 0.552330, acc.: 75.00%] [G loss: 0.932028]
29518 [D loss: 0.684547, acc.: 62.50%] [G loss: 0.934002]
29519 [D loss: 0.631660, acc.: 57.81%] [G loss: 0.935369]
29520 [D loss: 0.675999, acc.: 59.38%] [G loss: 1.040802]
29521 [D loss: 0.573253, acc.: 65.62%] [G loss: 1.088865]
29522 [D loss: 0.616762, acc.: 68.75%] [G loss: 1.030561]
29523 [D loss: 0.730147, acc.: 51.56%] [G loss: 0.943870]
29524 [D loss: 0.588731, acc.: 64.06%] [G loss: 1.037368]
29525 [D loss: 0.645326, acc.: 64.06%] [G loss: 1.111778]
29526 [D loss: 0.619737, acc.: 57.81%] [G loss: 1.142811]
29527 [D loss: 0.620104, acc.: 65.62%] [G loss: 1.117039]
29528 [D loss: 0.670674, acc.: 60.94%] [G loss: 0.989914]
29529 [D loss: 0.625905, acc.: 62.50%] [G loss: 0.877105]
29530 [D loss:

29656 [D loss: 0.629430, acc.: 65.62%] [G loss: 1.107435]
29657 [D loss: 0.640199, acc.: 60.94%] [G loss: 0.996142]
29658 [D loss: 0.627584, acc.: 67.19%] [G loss: 1.061738]
29659 [D loss: 0.630038, acc.: 59.38%] [G loss: 1.046294]
29660 [D loss: 0.675161, acc.: 59.38%] [G loss: 1.053814]
29661 [D loss: 0.630921, acc.: 65.62%] [G loss: 1.062497]
29662 [D loss: 0.631988, acc.: 65.62%] [G loss: 1.057884]
29663 [D loss: 0.637693, acc.: 64.06%] [G loss: 0.940285]
29664 [D loss: 0.611540, acc.: 60.94%] [G loss: 1.064161]
29665 [D loss: 0.716663, acc.: 50.00%] [G loss: 1.002119]
29666 [D loss: 0.626209, acc.: 64.06%] [G loss: 0.994295]
29667 [D loss: 0.668539, acc.: 62.50%] [G loss: 0.924302]
29668 [D loss: 0.630197, acc.: 64.06%] [G loss: 1.021573]
29669 [D loss: 0.525216, acc.: 79.69%] [G loss: 0.850592]
29670 [D loss: 0.651149, acc.: 57.81%] [G loss: 0.983440]
29671 [D loss: 0.557286, acc.: 70.31%] [G loss: 1.045849]
29672 [D loss: 0.690574, acc.: 57.81%] [G loss: 1.123787]
29673 [D loss:

29800 [D loss: 0.622401, acc.: 67.19%] [G loss: 1.076814]
29801 [D loss: 0.536936, acc.: 71.88%] [G loss: 1.146630]
29802 [D loss: 0.667908, acc.: 59.38%] [G loss: 1.020568]
29803 [D loss: 0.689597, acc.: 59.38%] [G loss: 0.961108]
29804 [D loss: 0.602182, acc.: 70.31%] [G loss: 1.138417]
29805 [D loss: 0.674312, acc.: 60.94%] [G loss: 1.139705]
29806 [D loss: 0.648355, acc.: 64.06%] [G loss: 0.958356]
29807 [D loss: 0.681817, acc.: 50.00%] [G loss: 0.961850]
29808 [D loss: 0.545993, acc.: 78.12%] [G loss: 1.101271]
29809 [D loss: 0.655275, acc.: 56.25%] [G loss: 1.016806]
29810 [D loss: 0.625265, acc.: 62.50%] [G loss: 1.223364]
29811 [D loss: 0.796622, acc.: 48.44%] [G loss: 1.197985]
29812 [D loss: 0.585296, acc.: 68.75%] [G loss: 1.114478]
29813 [D loss: 0.619275, acc.: 59.38%] [G loss: 1.104384]
29814 [D loss: 0.630745, acc.: 60.94%] [G loss: 1.075595]
29815 [D loss: 0.627755, acc.: 67.19%] [G loss: 0.928965]
29816 [D loss: 0.649100, acc.: 59.38%] [G loss: 1.009406]
29817 [D loss:

29945 [D loss: 0.637025, acc.: 59.38%] [G loss: 0.975498]
29946 [D loss: 0.728186, acc.: 53.12%] [G loss: 1.062010]
29947 [D loss: 0.642386, acc.: 64.06%] [G loss: 1.032099]
29948 [D loss: 0.635052, acc.: 65.62%] [G loss: 1.074695]
29949 [D loss: 0.586166, acc.: 68.75%] [G loss: 0.917662]
29950 [D loss: 0.611264, acc.: 64.06%] [G loss: 1.154268]
29951 [D loss: 0.628059, acc.: 62.50%] [G loss: 1.103200]
29952 [D loss: 0.667326, acc.: 60.94%] [G loss: 1.101416]
29953 [D loss: 0.659839, acc.: 67.19%] [G loss: 0.982369]
29954 [D loss: 0.669383, acc.: 59.38%] [G loss: 0.948000]
29955 [D loss: 0.610138, acc.: 64.06%] [G loss: 0.993786]
29956 [D loss: 0.604398, acc.: 68.75%] [G loss: 1.044253]
29957 [D loss: 0.687709, acc.: 59.38%] [G loss: 0.939995]
29958 [D loss: 0.687114, acc.: 53.12%] [G loss: 0.954227]
29959 [D loss: 0.604436, acc.: 65.62%] [G loss: 0.994812]
29960 [D loss: 0.577882, acc.: 68.75%] [G loss: 0.994654]
29961 [D loss: 0.673524, acc.: 64.06%] [G loss: 0.939057]
29962 [D loss: